# Extreme Event Analysis in ERA5 PV Variables


In [ ]:
import xarray as xr
import pandas as pd
import plotly.express as px
from scripts.analysis import extreme_events
ds = xr.open_dataset('data/processed/era5_processed_Bonn_2024_months_1-6.nc')
df = ds.to_dataframe().reset_index()


## Prolonged Low Irradiance Events

In [ ]:
events = extreme_events.find_prolonged_low_irradiance(df['surface_solar_radiation_downwards_w_m2'], threshold=50, min_duration=3)
fig = px.line(df, x='time', y='surface_solar_radiation_downwards_w_m2', title='Prolonged Low Irradiance Events')
for start, end in events:
    fig.add_vrect(x0=df['time'][start], x1=df['time'][end], fillcolor='red', opacity=0.2, line_width=0)
fig.show()


## Heatwaves

In [ ]:
events = extreme_events.find_heatwaves(df['2m_temperature_c'], threshold=30, min_duration=3)
fig = px.line(df, x='time', y='2m_temperature_c', title='Heatwaves')
for start, end in events:
    fig.add_vrect(x0=df['time'][start], x1=df['time'][end], fillcolor='orange', opacity=0.2, line_width=0)
fig.show()


## High Cloud Cover Events

In [ ]:
events = extreme_events.find_high_cloud_cover(df['total_cloud_cover'], threshold=0.8, min_duration=3)
fig = px.line(df, x='time', y='total_cloud_cover', title='High Cloud Cover Events')
for start, end in events:
    fig.add_vrect(x0=df['time'][start], x1=df['time'][end], fillcolor='blue', opacity=0.2, line_width=0)
fig.show()
